In [1]:
import os

import sys
sys.path.append('..')

import toml

import matplotlib.pyplot as plt

import torch
import numpy as np

from models.vision_transformer import ViT
from data.datasets import TimeSeriesDataset
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

In [2]:
config_path = '/glade/u/home/jshen/pruning-turbulence-vit/src/config/prune/l1.toml'
config = toml.load(config_path)
config['checkpoint_file'] = '/glade/derecho/scratch/jshen/base/best.tar'

model = ViT(**config['model'])
state_dict = torch.load(config['checkpoint_file'], map_location='cpu', weights_only=False)
model.load_state_dict(state_dict)
model.eval()

num_rollout_steps = config['finetuning']['num_rollout_steps']

config['train_dataset']['target_step'] *= num_rollout_steps
print('Target step:', config['train_dataset']['target_step'])
dataset = TimeSeriesDataset(**config['train_dataset'])
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

Target step: 9


In [14]:
i = 0

x, y_true = dataset[i]
ic = x.unsqueeze(0)
y_true = y_true.unsqueeze(0)

rollout = []
for i in range(num_rollout_steps):
    y_pred = model(ic)
    ic = torch.cat([y_pred, ic[:, :, :-1, :, :]], dim=2)
    rollout.append(y_pred)

torch.nn.functional.mse_loss(y_pred, y_true).backward()